In [32]:
    import pandas as pd
    import numpy as np
    import tensorflow as tf
    import shap
    from keras.models import Sequential
    from keras.layers import LSTM, Dense

    # Load CSV data
    data = pd.read_csv('AAPL_2006-01-01_to_2018-01-01.csv')

    # Preprocessing
    # Assuming you have columns like 'feature1', 'feature2', ..., 'target'
    X = data[['Close', 'Open']].values
    y = data['Open'].values

    # Define the number of time steps in your sequences
    n_timesteps = 10  # Change this value to your desired number of time steps

    # Reshape X into sequences of length n_timesteps
    X_sequences = []
    for i in range(n_timesteps, len(X)):
        X_sequences.append(X[i - n_timesteps:i])
    X_sequences = np.array(X_sequences)

    # Reshape y to align with X_sequences
    y = y[n_timesteps:]
    
    print(X_sequences)

    # Split data
    split_idx = int(0.8 * len(X_sequences))
    X_train, X_test = X_sequences[:split_idx], X_sequences[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]

    # Build and train LSTM model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_timesteps, X.shape[1])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=10, batch_size=32)

    # Load the trained model
    loaded_model = tf.keras.models.load_model('LSTM Prediction model.h5')

    # Load a specific test instance for explanation
    test_instance = X_test[0:1]
    
    test_instance.dtype()

    # Initialize SHAP explainer
    explainer = shap.Explainer(model, X_train)

    # Compute SHAP values
    shap_values = explainer(test_instance)

    # Plot the SHAP summary plot
    shap.summary_plot(shap_values, test_instance, feature_names=['feature1', 'feature2'])


[[[ 10.68  10.34]
  [ 10.71  10.73]
  [ 10.63  10.69]
  ...
  [ 12.04  12.14]
  [ 12.23  12.14]
  [ 12.1   12.24]]

 [[ 10.71  10.73]
  [ 10.63  10.69]
  [ 10.9   10.75]
  ...
  [ 12.23  12.14]
  [ 12.1   12.24]
  [ 11.78  11.87]]

 [[ 10.63  10.69]
  [ 10.9   10.75]
  [ 10.86  10.96]
  ...
  [ 12.1   12.24]
  [ 11.78  11.87]
  [ 11.29  11.61]]

 ...

 [[171.7  172.15]
  [172.27 172.5 ]
  [172.22 172.4 ]
  ...
  [175.01 174.17]
  [175.01 174.68]
  [170.57 170.8 ]]

 [[172.27 172.5 ]
  [172.22 172.4 ]
  [173.97 173.63]
  ...
  [175.01 174.68]
  [170.57 170.8 ]
  [170.6  170.1 ]]

 [[172.22 172.4 ]
  [173.97 173.63]
  [176.42 174.88]
  ...
  [170.57 170.8 ]
  [170.6  170.1 ]
  [171.08 171.  ]]]
Epoch 1/10
76/76 [==============================] - 0s 2ms/step - loss: 2704.8892
Epoch 2/10
76/76 [==============================] - 0s 2ms/step - loss: 3.3850
Epoch 3/10
76/76 [==============================] - 0s 2ms/step - loss: 3.2352
Epoch 4/10
76/76 [==============================] - 0s 2ms

TypeError: 'numpy.dtype[float64]' object is not callable